In [1]:
import pickle
import librosa
import numpy as np
import os

from tkinter import *

import time
 

In [2]:
# load the model 
filename = 'trained_model.sav'
load_model =pickle.load(open(filename, 'rb'))
load_model.weights

Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[<tf.Variable 'conv2d_7/kernel:0' shape=(5, 5, 1, 24) dtype=float32_ref>,
 <tf.Variable 'conv2d_7/bias:0' shape=(24,) dtype=float32_ref>,
 <tf.Variable 'conv2d_8/kernel:0' shape=(5, 5, 24, 48) dtype=float32_ref>,
 <tf.Variable 'conv2d_8/bias:0' shape=(48,) dtype=float32_ref>,
 <tf.Variable 'conv2d_9/kernel:0' shape=(5, 5, 48, 48) dtype=float32_ref>,
 <tf.Variable 'conv2d_9/bias:0' shape=(48,) dtype=float32_ref>,
 <tf.Variable 'dense_5/kernel:0' shape=(2400, 64) dtype=float32_ref>,
 <tf.Variable 'dense_5/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense_6/kernel:0' shape=(64, 5) dtype=float32_ref>,
 <tf.Variable 'dense_6/bias:0' shape=(5,) dtype=float32_ref>]

In [3]:
# load the encoder
with open("encoderX.txt", "rb") as fp:   # Unpickling
    encoderX = pickle.load(fp)

encoderX

[['Cheers', 0],
 ['Drinking', 1],
 ['Opening _ Closing', 2],
 ['Pouring', 3],
 ['Shaking', 4]]

In [4]:

def make_prediction(path):

    D = [] # Dataset

    for ID, filename in enumerate(os.listdir(path)):
        if filename.endswith(".wav") or filename.endswith(".mps") or filename.endswith(".m4a"): 
            if os.path.exists(path+ '/' + filename):
                y, sr = librosa.load(path+ '/' + filename, duration=2.97)
                if len(y)>=65489: # ignore the too short sound
                    ps = librosa.feature.melspectrogram(y=y, sr=sr)
                    D.append( (ps, 0) )
        
    X_test, y_test = zip(*D)

    X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])

    # make the predict
    y_load_predict=load_model.predict(X_test)

    y_code = y_load_predict.argmax(axis=1)
    y_code_ratio = y_load_predict.max(axis=1)

    y_label = []

    # make the possibility<60% to 'others'
    for ind, code in enumerate(y_code):
        if y_code_ratio[ind]<0.6:
            y_label.append(['others', 10])
        else:
            y_label.append(encoderX[code])

    return y_label

In [ ]:
my_window = Tk()
width_value = my_window.winfo_screenwidth()
height_value = my_window.winfo_screenheight()
my_window.geometry("%dx%d+0+0" % (width_value, height_value))

T = Text(my_window, height=int(height_value), width=int(width_value))
T.pack()
T.tag_configure('bold_italics', font=('Arial', 12, 'bold', 'italic'))
T.tag_configure('big', font=('Verdana', 50, 'bold'))
    
while 1:
    y_label = make_prediction('Test_Set')
    T.delete('1.0', END)
    for c in y_label:
        T.insert(END, c[0], 'big')
        T.insert(END, '\n')
    my_window.update()
    # Wait for 1 seconds
    time.sleep(1)